In [17]:
import util
from backtesting import Backtest
import pandas_ta as ta
import numpy as np

lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'DOGEUSDT', 'DOTUSDT', 
         'SOLUSDT', 'AVAXUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'OPUSDT', 'FETUSDT', 'AGIXUSDT', 'ARBUSDT',  
         'SLPUSDT', 'MEMEUSDT',  '1000SATSUSDT', 'PIXELUSDT']
lista=['BTCUSDT']

suma=0.0
trades = 0
timeframe='1h'
limit= 1000

print(f"timeframe: {timeframe} - limit: {limit}")
for symbol in lista:
    try:
        data=util.obtiene_historial(symbol=symbol,timeframe=timeframe,limit=limit)    
        data['ema100']=ta.ema(data.Close, length=100)
        data['ema200']=ta.ema(data.Close, length=200)
        porcentaje_perdida = 1
        ##    obligatorios   --->>>  trade, stop_loss, take_profit, porcentajeentrada
        data['trade'] = np.where(util.crossover_dataframe(data.ema100, data.ema200),-1,np.where(util.crossover_dataframe(data.ema200, data.ema100),-2,0))
        data['stop_loss'] = np.where(data.trade==1,data.ema200,np.where(data.trade==-1,data.ema200,0))
        variacion_hasta_stop_loss = abs(((data.stop_loss/data.Close)-1)*100)
        data['take_profit'] = np.where(data.trade==1,data.Close+(data.Close-data.ema200)*3,np.where(data.trade==-1,data.Close-(data.ema200-data.Close)*3,0))           
        data['porcentajeentrada'] = np.where(((porcentaje_perdida/variacion_hasta_stop_loss))>=1,0.99,((porcentaje_perdida/variacion_hasta_stop_loss)))        ##
        ###
        bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
        output = bt.run()
        if len(lista)==1:
            bt.plot(filename="graficos/"+symbol)
        print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
        suma=suma+output['Return [%]']
        trades=trades+output['# Trades']
    except Exception as e:
        print(f"Error en {symbol}: {e}")
print(f"Suma total: {suma} - Trades total: {trades}")

timeframe: 1h - limit: 1000

Error: Column 'Indicator1' not in data - line: 1140 - file: backtesting.py
Error en BTCUSDT: First issue `backtest.run()` to obtain results.
Suma total: 0.0 - Trades total: 0


In [3]:
import util
from backtesting import Backtest
import pandas_ta as ta
import numpy as np

lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'DOGEUSDT', 'DOTUSDT', 
         'SOLUSDT', 'AVAXUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'OPUSDT', 'FETUSDT', 'AGIXUSDT', 'ARBUSDT',  
         'SLPUSDT', 'MEMEUSDT',  '1000SATSUSDT', 'PIXELUSDT']
#lista=['BTCUSDT']

suma=0.0
trades = 0
timeframe='1h'
limit= 1000

print(f"timeframe: {timeframe} - limit: {limit}")
for symbol in lista:
    try:
        data=util.obtiene_historial(symbol=symbol,timeframe=timeframe,limit=limit)    
        data['sma200']=ta.sma(data.Close, length=200)
        porcentaje_perdida = 1
        data['sma5'] = ta.sma(data.Close, length=5)
        ##    obligatorios   --->>>  trade, stop_loss, take_profit, porcentajeentrada
        data['trade'] = np.where((data.Close.shift(3) > data.Close.shift(2)) & (data.Close.shift(2) > data.Close.shift(1))                                 
                                 & (data.Close > data.sma200)
                                 & (data.sma200 > data.sma200.shift(1))
                                 & (data.sma5 > data.Close)
                                 ,1,0)
        data['stop_loss'] = data.Low
        variacion_hasta_stop_loss = abs(((data.stop_loss/data.Close)-1)*100)
        data['take_profit'] =  data['sma5']
        data['porcentajeentrada'] = 0.1 #np.where(((porcentaje_perdida/variacion_hasta_stop_loss))>=1,0.99,((porcentaje_perdida/variacion_hasta_stop_loss)))        ##
        ###
        bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
        output = bt.run()
        if len(lista)==1:
            bt.plot(filename="graficos/"+symbol)
        print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
        suma=suma+output['Return [%]']
        trades=trades+output['# Trades']
    except Exception as e:
        print(f"Error en {symbol}: {e}")
print(f"Suma total: {suma} - Trades total: {trades}")

timeframe: 1h - limit: 1000
Symbol: BTCUSDT - Return [%]: 0.03 - # Trades: 36 - Win Rate [%]: 55.55555555555556
Symbol: ETHUSDT - Return [%]: -0.22 - # Trades: 19 - Win Rate [%]: 31.57894736842105

Error: Long orders require: SL (411.07) < LIMIT (411.07) < TP (417.98200000000014) - line: 1171 - file: backtesting.py
Error en BCHUSDT: 'NoneType' object is not subscriptable
Symbol: XRPUSDT - Return [%]: 0.51 - # Trades: 45 - Win Rate [%]: 53.333333333333336
Symbol: LTCUSDT - Return [%]: 0.01 - # Trades: 22 - Win Rate [%]: 36.36363636363637
Symbol: ETCUSDT - Return [%]: 0.29 - # Trades: 19 - Win Rate [%]: 47.368421052631575
Symbol: LINKUSDT - Return [%]: 0.14 - # Trades: 16 - Win Rate [%]: 56.25
Symbol: ADAUSDT - Return [%]: -0.29 - # Trades: 18 - Win Rate [%]: 33.33333333333333
Symbol: BNBUSDT - Return [%]: 0.17 - # Trades: 44 - Win Rate [%]: 47.72727272727273
Symbol: DOGEUSDT - Return [%]: -0.41 - # Trades: 25 - Win Rate [%]: 36.0
Symbol: DOTUSDT - Return [%]: -0.16 - # Trades: 8 - Win R

In [5]:
#divergencias


import util
from backtesting import Backtest
import pandas_ta as ta
import numpy as np
from sklearn.preprocessing import MinMaxScaler

lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'DOGEUSDT', 'DOTUSDT', 
         'SOLUSDT', 'AVAXUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'OPUSDT', 'FETUSDT', 'AGIXUSDT', 'ARBUSDT',  
         'SLPUSDT', 'MEMEUSDT',  '1000SATSUSDT', 'PIXELUSDT']
#lista=['BTCUSDT']

suma=0.0
trades = 0
timeframe='1d'
limit= 1000
scaler = MinMaxScaler()

print(f"timeframe: {timeframe} - limit: {limit}")
for symbol in lista:
    try:
        data = util.obtiene_historial(symbol=symbol,timeframe=timeframe,limit=limit)    
        data['sma'] = ta.sma(data.Close, length = 50) ##se puede variar
        data['macd'] = ta.macd(data['Close'])['MACD_12_26_9']
        data['sma_macd'] = ta.sma(data.macd, length=10) ##se puede variar
        # Escalar los indicadores al rango del precio
        data[['Indicator1', 'Indicator2']] = scaler.fit_transform(data[['sma', 'sma_macd']])
        price_range = data['Close'].max() - data['Close'].min()
        data['Indicator1'] = data['Indicator1'] * price_range + data['Close'].min()
        data['Indicator2'] = data['Indicator2'] * price_range + data['Close'].min()
        porcentaje_perdida = 1
        ##################################    obligatorios ##############  --->>>  trade, stop_loss, take_profit, porcentajeentrada
        ###########################################################################################################################
        data['trade'] = np.where(util.crossover_dataframe(data.Indicator2, data.Indicator1),-1,np.where(util.crossover_dataframe(data.Indicator1, data.Indicator2),-2,-1.5))
        data['stop_loss'] = np.where(data.trade==-1,data.Close-data.atr*3,np.where(data.trade==-2,data.Close+data.atr*3,0))
        variacion_hasta_stop_loss = abs(((data.stop_loss/data.Close)-1)*100)
        data['take_profit'] = None #np.where(data.trade==1,data.Close+data.atr*6,np.where(data.trade==-2,data.Close-data.atr*6,0))
        data['porcentajeentrada'] = np.where(((porcentaje_perdida/variacion_hasta_stop_loss))>=1,0.99,((porcentaje_perdida/variacion_hasta_stop_loss)))        ##
        ###
        bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
        output = bt.run()
        if len(lista)==1:
            bt.plot(filename="graficos/"+symbol)
        print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
        suma=suma+output['Return [%]']
        trades=trades+output['# Trades']
    except Exception as e:
        print(f"Error en {symbol}: {e}")
print(f"Suma total: {suma} - Trades total: {trades}")

timeframe: 1d - limit: 1000
Symbol: BTCUSDT - Return [%]: 0.46 - # Trades: 4 - Win Rate [%]: 25.0
Symbol: ETHUSDT - Return [%]: 5.51 - # Trades: 3 - Win Rate [%]: 66.66666666666666
Symbol: BCHUSDT - Return [%]: 7.58 - # Trades: 7 - Win Rate [%]: 57.14285714285714
Symbol: XRPUSDT - Return [%]: 4.6 - # Trades: 4 - Win Rate [%]: 75.0
Symbol: LTCUSDT - Return [%]: 2.07 - # Trades: 4 - Win Rate [%]: 75.0
Symbol: ETCUSDT - Return [%]: 3.13 - # Trades: 4 - Win Rate [%]: 50.0
Symbol: LINKUSDT - Return [%]: -1.31 - # Trades: 9 - Win Rate [%]: 22.22222222222222
Symbol: ADAUSDT - Return [%]: 0.63 - # Trades: 3 - Win Rate [%]: 66.66666666666666
Symbol: BNBUSDT - Return [%]: 9.3 - # Trades: 3 - Win Rate [%]: 66.66666666666666
Symbol: DOGEUSDT - Return [%]: 5.91 - # Trades: 1 - Win Rate [%]: 100.0
Symbol: DOTUSDT - Return [%]: -1.43 - # Trades: 2 - Win Rate [%]: 0.0
Symbol: SOLUSDT - Return [%]: -0.91 - # Trades: 4 - Win Rate [%]: 50.0
Symbol: AVAXUSDT - Return [%]: 2.0 - # Trades: 3 - Win Rate [%]: